__CalCOFI Hackathon Basic Framework Tester__

Packages:

In [92]:
import numpy as np  # provides multi-dimensional array and matrices functionality
import pandas as pd  # provides interface for interacting with tabular data
import matplotlib.pyplot as plt  # provides plotting tools
import seaborn as sns  # visualization tools
%matplotlib inline
from scipy.ndimage.filters import gaussian_filter
# %conda install folium -c conda-forge
import folium  # gis visualization
from folium import plugins
from folium.plugins import HeatMap  # heatmap visualization package

Cleaning Bottle Data:

In [93]:
cc = pd.read_csv("C:/Users/eudor/Downloads/bottle.csv/bottle.csv")  # CalCOFI bottle data

surface = cc[cc['Depthm'] == 0]  # select surface level data

aspects = surface[['Sta_ID', 'Depth_ID', 'T_degC', 'Salnty', 'O2ml_L', 'ChlorA']].fillna(0)  # tentative data layers separation, replace NaNs with 0

yymm = list(aspects["Depth_ID"])  # using Depth_ID to extract year and month, add to dataframe
year = []
month = []
for i in yymm:
    yr = int(i[:2] + i[3:5])
    mon = int(i[5:7])
    year.append(yr)
    month.append(mon)

aspects.insert(1,"Year", year, True)
aspects.insert(2, "Month", month, True)
del aspects["Depth_ID"]

aspects_15 = aspects[aspects['Year']==2015]  # selecting April of 2015
spring_15 = aspects_15[aspects_15['Month']==4]
spring_15

,Sta_ID,Year,Month,T_degC,Salnty,O2ml_L,ChlorA
850578,076.7 049.0,2015,4,11.510,33.5598,7.114,22.632
850588,076.7 051.0,2015,4,12.675,33.3538,6.647,2.522
850608,076.7 055.0,2015,4,13.669,33.2469,6.210,0.717
850638,076.7 060.0,2015,4,14.151,33.2475,6.320,1.537
850666,076.7 070.0,2015,4,15.318,33.1735,5.876,0.271
...,...,...,...,...,...,...,...
852218,093.3 090.0,2015,4,17.518,33.1430,5.647,0.073
852251,093.3 100.0,2015,4,17.338,33.2920,5.593,0.086
852281,093.3 110.0,2015,4,17.988,33.5507,5.483,0.068
852313,093.3 120.0,2015,4,17.792,33.3277,5.563,0.058


Station ID Coordinate Data Manipulation:

In [94]:
coords = pd.read_csv("C:/Users/eudor/Downloads/CalCOFIStationOrder.csv")  # CalCOFI Station Order Data
lat_lon = coords[['Line', 'Sta', 'Lat (dec)', 'Lon (dec)']]  # isolate columns
lat_lon

,Line,Sta,Lat (dec),Lon (dec)
0,93.3,26.7,32.95637,-117.30538
1,93.4,26.4,32.94905,-117.27357
2,91.7,26.4,33.24350,-117.46542
3,93.3,28.0,32.91304,-117.39438
4,93.3,30.0,32.84637,-117.53122
...,...,...,...,...
108,60.0,60.0,37.61410,-123.60825
109,60.0,70.0,37.28077,-124.33237
110,60.0,80.0,36.94743,-125.05327
111,60.0,90.0,36.61410,-125.77099


Bottle Data & Coordinates Merged:

In [95]:
line = []  #separating line and station in bottle data to match coordinate data
station = []
sta_id = list(spring_15['Sta_ID'])
for i in sta_id:
    li = float(i[:5])
    sta = float(i[6:])
    line.append(li)
    station.append(sta)
    
spring_15.insert(0,"Line", line, True)
spring_15.insert(1, "Sta", station, True)

merged = pd.merge(spring_15, lat_lon, how='left', left_on=['Line','Sta'], right_on = ['Line','Sta']) #merge bottle and coordinate data
merged

,Line,Sta,Sta_ID,Year,Month,T_degC,Salnty,O2ml_L,ChlorA,Lat (dec),Lon (dec)
0,76.7,49.0,076.7 049.0,2015,4,11.510,33.5598,7.114,22.632,35.08824,-120.77740
1,76.7,51.0,076.7 051.0,2015,4,12.675,33.3538,6.647,2.522,35.02158,-120.91782
2,76.7,55.0,076.7 055.0,2015,4,13.669,33.2469,6.210,0.717,34.88824,-121.19831
3,76.7,60.0,076.7 060.0,2015,4,14.151,33.2475,6.320,1.537,34.72158,-121.54828
4,76.7,70.0,076.7 070.0,2015,4,15.318,33.1735,5.876,0.271,34.38824,-122.24608
...,...,...,...,...,...,...,...,...,...,...,...
65,93.3,90.0,093.3 090.0,2015,4,17.518,33.1430,5.647,0.073,30.84637,-121.58965
66,93.3,100.0,093.3 100.0,2015,4,17.338,33.2920,5.593,0.086,30.51304,-122.25760
67,93.3,110.0,093.3 110.0,2015,4,17.988,33.5507,5.483,0.068,30.17971,-122.92324
68,93.3,120.0,093.3 120.0,2015,4,17.792,33.3277,5.563,0.058,29.84637,-123.58661


Visualization:

In [108]:
m = folium.Map(location=[32.88719,-117.2546], tiles="Stamen Terrain")  # general mapping

for i in range(len(list(merged['Lat (dec)']))):  # station markers
    folium.Marker(location=[list(merged['Lat (dec)'])[i],list(merged['Lon (dec)'])[i]], popup=list(merged['Sta_ID'])[i], icon=folium.Icon(color="red", icon="info-sign")).add_to(m)

heat_data = [[row['Lat (dec)'],row['Lon (dec)'],row['T_degC']] for index, row in merged.iterrows()]

# Plot it on the map
HeatMap(heat_data,name=None, min_opacity=0.3, max_zoom=18, radius=100, blur=30, gradient=None, overlay=True, control=True, show=False).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

# Display the map
m